In [1]:
import numpy as np
from bokeh.plotting import figure, show, output_file , output_notebook

from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.plotting import figure, show
from bokeh.plotting import output_notebook
from bokeh.models import WheelZoomTool
from longi_debug_xviz import *
from pprint import pprint
from common.bag_reader import * 
from common.figure_viz import * 
from common.layer_viz import * 
from common.plot_viz import * 

In [9]:
data_set = [{'x':-35, 'y':-5},{'x':-30, 'y':-10},{'x':-25, 'y':-15}]

data_set2 = {"x":[-35, -30, -25], "y":[-5, -10, -15]}

def transform_dataset(data_set):
    if not data_set:
        return {}

    # 获取所有键
    keys = data_set[0].keys()
    
    # 初始化新的数据集格式
    transformed = {key: [] for key in keys}

    # 遍历原始数据集，收集每个键的值
    for item in data_set:
        for key in keys:
            transformed[key].append(item[key])

    return transformed

# 测试函数
data_set = [{'x': -35, 'y': -5}, {'x': -30, 'y': -10}, {'x': -25, 'y': -15}]
transformed_data_set = transform_dataset(data_set)
print(transformed_data_set)


{'x': [-35, -30, -25], 'y': [-5, -10, -15]}


In [2]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Slider, CustomJS
from bokeh.layouts import column, row
import pandas as pd

# heading 数据示例
traj_vel = [{'data': 10, 'index': 0, 't': 0},
            {'data': 20, 'index': 1, 't': 0.1543},
            {'data': 50, 'index': 2, 't': 0.2532},
            {'data': 20, 'index': 3, 't': 0.3576},
            {'data': 80, 'index': 4, 't': 0.4543},
            {'data': 40, 'index': 5, 't': 0.5564}
            ]

traj_vel = [{'data': 10, 'index': 0, 't': 0},
            {'data': 20, 'index': 1, 't': 0.1543},
            {'data': 50, 'index': 2, 't': 0.2532},
            {'data': 20, 'index': 3, 't': 0.3576},
            {'data': 80, 'index': 4, 't': 0.4543},
            {'data': 40, 'index': 5, 't': 0.5564}
            ]

df_heading = pd.DataFrame(traj_vel)


source_heading = ColumnDataSource(data=dict(index=df_heading['index'], data=df_heading['data'], 
                                            color=['blue']*len(df_heading), size=[5]*len(df_heading)))

plot_heading = figure(title='Heading', x_axis_label='Index', y_axis_label='Data',width=400 ,height = 300)
plot_heading.line('index', 'data', source=source_heading)
plot_heading.circle('index', 'data', source=source_heading, color='color', size='size')


times = df_heading['t'].tolist()


callback_code = """
    const data_heading = source_heading.data;
    const t = cb_obj.value;
    let closest_index = 0;
    let minDiff = Math.abs(times[0] - t);
    for (let i = 1; i < times.length; i++) {
        let diff = Math.abs(times[i] - t);
        if (diff < minDiff) {
            minDiff = diff;
            closest_index = i;
        }
    }
    
    for (let i = 0; i < data_heading.index.length; i++) {
        if (data_heading.index[i] === Math.round(times[closest_index] * 10)) {
            data_heading.color[i] = 'red';
            data_heading.size[i] = 10;
        } else {
            data_heading.color[i] = 'blue';
            data_heading.size[i] = 5;
        }
    }

    source_heading.change.emit();
"""
callback = CustomJS(args=dict(source_heading=source_heading, times=times), code=callback_code)

# 滑块
slider = Slider(start=0, end=df_heading['t'].max(), value=0, step=0.1, title="Time")
slider.js_on_change('value', callback)

layout = column(slider, row(plot_heading))
output_notebook()
show(layout)


Loading BokehJS ...

In [5]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Slider, CustomJS
from bokeh.layouts import column
import pandas as pd

# ego_polygon 数据示例
ego_polygon1 = [
    {'data': [[0.1, 0.2], [0.2, 0.2], [0.2, 0.1], [0.1, 0.1]], 'index': 0, 't': 0},
    {'data': [[0.2, 0.3], [0.3, 0.3], [0.3, 0.2], [0.2, 0.2], [0.15, 0.25]], 'index': 1, 't': 0.1043},
    {'data': [[0.3, 0.4], [0.4, 0.4], [0.4, 0.3]], 'index': 2, 't': 0.2032},
    {'data': [[0.5, 0.3], [0.2, 0.7], [0.3, 0.3]], 'index': 3, 't': 0.3032},

]

ego_polygon2 = [
    {'data': [[10, 20], [20, 20], [20, 10], [10, 10], [15, 15], [17, 15], [16, 18]], 'index': 0, 't': 0},
    {'data': [[20, 30], [30, 30], [30, 20], [20, 20], [15, 25]], 'index': 1, 't': 0.1043},
    {'data': [[30, 40], [40, 40], [40, 30]], 'index': 2, 't': 0.2032},
    {'data': [[50, 30], [20, 70], [30, 30]], 'index': 3, 't': 0.3032},
]
# line 的点
line1 = [
    {'data': [[3, 3], [3, 4], [3, 5], [3, 6]], 'index': 0, 't': 0},
    {'data': [[4, 4], [4, 5], [4, 6], [4, 7], [4, 8]], 'index': 1, 't': 0.1043},
    {'data': [[5, 5], [5, 6], [5, 7]], 'index': 2, 't': 0.2032},
    {'data': [[6, 6], [6, 7], [6, 8]], 'index': 3, 't': 0.3032},
]

line2 = [
    {'data': [[4, 3], [4, 4], [5, 5], [6, 6]], 'index': 0, 't': 0},
    {'data': [[5, 4], [5, 5], [6, 6], [7, 7], [8, 8]], 'index': 1, 't': 0.1043},
    {'data': [[6, 5], [6, 6], [7, 7]], 'index': 2, 't': 0.2032},
    {'data': [[7, 6], [7, 7], [8, 8]], 'index': 3, 't': 0.3032},
]

line3 = [
    {'data': {"x":[4,4,5,6], "y":[3,4,5,6]}, 'index': 0, 't': 0},
    {'data': {"x":[5,5,6,7,8], "y":[4,5,6,7,8]}, 'index': 1, 't': 0.1043},
    {'data': {"x":[6,6,7], "y":[5,6,7]}, 'index': 2, 't': 0.2032},
    {'data': {"x":[7,7,8], "y":[6,7,8]}, 'index': 3, 't': 0.3032},
]

leader_s = [
    {'data': [10, 20, 30, 40], 'index': 0, 't': 0},
    {'data': [20, 30, 40, 50], 'index': 1, 't': 0.1043},
    {'data': [30, 40, 50, 60], 'index': 2, 't': 0.2032},
    {'data': [40, 50, 60, 70], 'index': 3, 't': 0.3032},
]

leader_v = [
    {'data': [10, 10, 10, 10], 'index': 0, 't': 0},
    {'data': [10, 10, 10, 20], 'index': 1, 't': 0.1043},
    {'data': [10, 10, 20, 20], 'index': 2, 't': 0.2032},
    {'data': [10, 20, 20, 20], 'index': 3, 't': 0.3032},
]

def combine_leader_data(leader_s, leader_v):
    leader_s_v = []
    for s, v in zip(leader_s, leader_v):
        combined_data = [[s_data, v_data] for s_data, v_data in zip(s['data'], v['data'])]
        leader_s_v.append({
            'data': combined_data,
            'index': s['index'],
            't': s['t']
        })
    return leader_s_v

# leader_s_v = [
#     {'data': [[10, 10], [20, 10], [30, 10], [40, 10]], 'index': 0, 't': 0},
#     {'data': [[20, 10], [30, 10], [40, 10], [50, 20]], 'index': 1, 't': 0.1043},
#     {'data': [[30, 10], [30, 10], [40, 20], [50, 20]], 'index': 2, 't': 0.2032},
#     {'data': [[40, 10], [50, 20], [60, 20], [70, 20]], 'index': 3, 't': 0.3032},
# ]

leader_s_v = combine_leader_data(leader_s, leader_v)

def create_polygon_data_sources(datasets):
    sources = []
    all_times = []
    all_poly_data = []

    for data in datasets:
        df = pd.DataFrame(data)
        times = df['t'].tolist()
        poly_data = df['data'].tolist()

        initial_xs, initial_ys = zip(*poly_data[0])
        source = ColumnDataSource(data=dict(xs=list(initial_xs), ys=list(initial_ys)))

        sources.append(source)
        all_times.append(times)
        all_poly_data.append(poly_data)

    return sources, all_times, all_poly_data

def create_line_data_sources(line_datasets):
    sources = []
    all_times = []
    all_line_data = []

    for line_data in line_datasets:
        df = pd.DataFrame(line_data)
        times = df['t'].tolist()
        line_points = df['data'].tolist()

        initial_xs, initial_ys = zip(*line_points[0])
        source = ColumnDataSource(data=dict(xs=list(initial_xs), ys=list(initial_ys)))

        sources.append(source)
        all_times.append(times)
        all_line_data.append(line_points)

    return sources, all_times, all_line_data

datasets = [ego_polygon1, ego_polygon2]  # 包含所有polygon数据集

sources, all_times, all_poly_data = create_polygon_data_sources(datasets)



# 创建图表
plot = figure(title='Ego Polygons', x_axis_label='X', y_axis_label='Y', width=400, height=300)
colors = ['blue', 'green']
for source, color in zip(sources, colors):
    plot.patch('xs', 'ys', source=source, alpha=0.5, line_width=2, color=color)



def create_polygon_callback(sources, all_times, all_poly_data):
    callback_code = """
        const t = cb_obj.value;
        sources.forEach((source, index) => {
            const data = source.data;
            const times = all_times[index];
            const poly_data = all_poly_data[index];

            let closest_index = 0;
            let minDiff = Math.abs(times[0] - t);
            for (let i = 1; i < times.length; i++) {
                let diff = Math.abs(times[i] - t);
                if (diff < minDiff) {
                    minDiff = diff;
                    closest_index = i;
                }
            }

            let new_xs = [];
            let new_ys = [];
            for (let i = 0; i < poly_data[closest_index].length; i++) {
                new_xs.push(poly_data[closest_index][i][0]);
                new_ys.push(poly_data[closest_index][i][1]);
            }
            data.xs = new_xs;
            data.ys = new_ys;

            source.change.emit();
        });
    """
    return CustomJS(args=dict(sources=sources, all_times=all_times, 
                              all_poly_data=all_poly_data), code=callback_code)

line_datasets = [line1, line2, leader_s_v]  # 包含所有line数据集

all_line_sources, all_line_times, all_line_data = create_line_data_sources(line_datasets)

colors = ['red', 'green', 'black']
for source, color in zip(all_line_sources, colors):
    plot.scatter('xs', 'ys', source=source, line_width=2, color=color)

def create_line_callback(all_line_sources, all_line_times, all_line_data):
    callback_code = """
        const t = cb_obj.value;
        all_line_sources.forEach((source, index) => {
            const line_data = source.data;
            const times = all_line_times[index];
            const line_points = all_line_data[index];
            let new_xs = [];
            let new_ys = [];
            let closest_index = 0;
            let minDiff = Math.abs(times[0] - t);
            for (let i = 1; i < times.length; i++) {
                let diff = Math.abs(times[i] - t);
                if (diff < minDiff) {
                    minDiff = diff;
                    closest_index = i;
                }
            }

            for (let i = 0; i < line_points[closest_index].length; i++) {
                new_xs.push(line_points[closest_index][i][0]);
                new_ys.push(line_points[closest_index][i][1]);
            }
            line_data.xs = new_xs;
            line_data.ys = new_ys;

            source.change.emit();
        });
    """
    return CustomJS(args=dict(all_line_sources=all_line_sources, 
                              all_line_times=all_line_times, 
                              all_line_data=all_line_data), code=callback_code)


# 滑块
callback_polygon = create_polygon_callback(sources, all_times, all_poly_data)
callback_line = create_line_callback(all_line_sources, all_line_times, all_line_data)
max_time = max(max(times) for times in all_times)
slider = Slider(start=0, end=max_time, value=0, step=0.1, title="Time")
slider.js_on_change('value', callback_polygon, callback_line)

layout = column(slider, plot)
output_notebook()
show(layout)


Loading BokehJS ...

In [ ]:
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, Slider, CustomJS
from bokeh.layouts import column, row

def create_figure(datasets, title, x_axis_label, y_axis_label, plot_types):
    plot = figure(title=title, x_axis_label=x_axis_label, y_axis_label=y_axis_label)
    sources = []

    for data, plot_type in zip(datasets, plot_types):
        source = ColumnDataSource(data)
        sources.append(source)

        if plot_type == 'line':
            plot.line('x', 'y', source=source)
        elif plot_type == 'circle':
            plot.circle('x', 'y', source=source, fill_color="white", size=8)
        elif plot_type == 'both':
            plot.line('x', 'y', source=source)
            plot.circle('x', 'y', source=source, fill_color="white", size=8)

    return plot, sources

def create_callback(sources, data_js_list, times):
    callback_code = """
        const t = cb_obj.value;
        let closest_index = 0;
        let minDiff = Math.abs(times[0] - t);
        for (let i = 1; i < times.length; i++) {
            let diff = Math.abs(times[i] - t);
            if (diff < minDiff) {
                minDiff = diff;
                closest_index = i;
            }
        }
        for (let j = 0; j < sources.length; j++) {
            const data = sources[j].data;
            data.y = data_js_list[j][closest_index];
            data.x = Array.from(Array(data.y.length).keys()); // 更新x以匹配y的长度
            sources[j].change.emit();
        }
    """
    return CustomJS(args=dict(sources=sources, data_js_list=data_js_list, 
                              times=times), code=callback_code)

# 示例数据
traj_acc = [{'data': [0.1, 0.2, 0.3, 0.4, 0.1, 0.2, 0.3, 0.4], 'index': 0, 't': 0},
            {'data': [0.2, 0.3, 0.4, 0.5, 0.1, 0.2, 0.3, 0.4], 'index': 1, 't': 0.1543},
            {'data': [0.3, 0.4, 0.2, 0.6], 'index': 2, 't': 0.2532},
            {'data': [0.3, 0.5, 0.5, 0.6, 0.3, 0.4, 0.5, 0.1], 'index': 3, 't': 0.3576},
            {'data': [0.6, 0.4, 0.5, 0.6, 0.5], 'index': 4, 't': 0.4543},
            {'data': [0.5, 0.4, 0.5, 0.6, 0.3, 0.4, 0.5, 0.1], 'index': 5, 't': 0.5564}
           ]
traj_acc2 = [{'data': [1, 2, 3, 4, ], 'index': 0, 't': 0},
             {'data': [2, 3, 4, 5, 1, 2, ], 'index': 1, 't': 0.1543},
             {'data': [3, 4, 2, 6, 4, 5, 1, 2,], 'index': 2, 't': 0.2532},
             {'data': [3, 5, 5, 6, 5, 1], 'index': 3, 't': 0.3576},
             {'data': [6, 4, 5, 6, 5, 4, 5, 6, 3, 4, 5, 1], 'index': 4, 't': 0.4543},
             {'data': [5, 4, 5, 6, 3, 4, 5, 1], 'index': 5, 't': 0.5564}
             ]

traj_vel = [{'data': [10, 20, 30, 40, ], 'index': 0, 't': 0},
             {'data': [2, 3, 4, 5, 10, 20, ], 'index': 1, 't': 0.1543},
             {'data': [3, 4, 20, 6, 40, 5, 10, 2,], 'index': 2, 't': 0.2532},
             {'data': [3, 5, 50, 6, 5, 1], 'index': 3, 't': 0.3576},
             {'data': [6, 4, 5, 60, 5, 4, 50, 6, 30, 40, 5, 1], 'index': 4, 't': 0.4543},
             {'data': [5, 4, 5, 6, 30, 40, 5, 1], 'index': 5, 't': 0.5564}
             ]

df_traj_acc = pd.DataFrame(traj_acc)
df_traj_acc2 = pd.DataFrame(traj_acc2)
df_traj_vel = pd.DataFrame(traj_vel)

# 准备数据
data_js_1 = [row['data'] for index, row in df_traj_acc.iterrows()]
data_js_2 = [row['data'] for index, row in df_traj_acc2.iterrows()]
data_js_3 = [row['data'] for index, row in df_traj_vel.iterrows()]
times = df_traj_acc['t'].tolist()

data_1 = {'x': list(range(len(df_traj_acc.iloc[0]['data']))), 'y': df_traj_acc.iloc[0]['data']}
data_2 = {'x': list(range(len(df_traj_acc2.iloc[0]['data']))), 'y': df_traj_acc2.iloc[0]['data']}
data_3 = {'x': list(range(len(df_traj_vel.iloc[0]['data']))), 'y': df_traj_vel.iloc[0]['data']}

# 创建两个图表和数据源
plot1, sources1 = create_figure([data_1, data_2], 'Trajectory Data 1', 'Index', 'Data', ['both', 'both'])
plot2, sources2 = create_figure([data_3], 'Trajectory Data 2', 'Index', 'Data', ['both'])

# 创建共享回调
callback = create_callback(sources1 + sources2, [data_js_1, data_js_2, data_js_3], times)

# 滑块控件
slider = Slider(start=0, end=df_traj_acc['t'].max(), value=0, step=0.1, title="Time")
slider.js_on_change('value', callback)

# 显示
layout = column(slider, row(plot1, plot2))
output_notebook()
show(layout)

# # 准备数据
# data_js_1 = [row['data'] for index, row in df_traj_acc.iterrows()]
# data_js_2 = [row['data'] for index, row in df_traj_acc2.iterrows()]
# times = df_traj_acc['t'].tolist()

# data_1 = {'x': list(range(len(df_traj_acc.iloc[0]['data']))), 
#           'y': df_traj_acc.iloc[0]['data']}
# data_2 = {'x': list(range(len(df_traj_acc2.iloc[0]['data']))), 
#           'y': df_traj_acc2.iloc[0]['data']}

# # 创建图表和数据源
# plot, sources = create_figure([data_1, data_2], 'Trajectory Data', 'Index', 'Data', ['both', 'both'])

# # 创建回调
# callback = create_callback(sources, [data_js_1, data_js_2], times)

# # 滑块控件
# slider = Slider(start=0, end=df_traj_acc['t'].max(), value=0, step=0.1, title="Time")
# slider.js_on_change('value', callback)

# # 显示
# layout = column(slider, plot)
# output_notebook()
# show(layout)


Loading BokehJS ...

In [3]:
bag_path = '/home/sensetime/data/xpilot/11_2023/1123/2023_11_23_22_19_28/test.00000.rsclbag'
html_path = '/home/sensetime/data/xpilot/11_2023/1123/2023_11_23_22_19_28/test.html'
plot = PlotXpilotViz(bag_path, html_path)
# plot = NewPlotBase(bag_path, html_path)
plot.add_required_channel(XDEBUG_CHANNEL)
# plot.add_required_channel(PLANNING_CHANNEL)
# plot.figure_plot()


In [4]:
plot.construct_msg()

plot.match_timestamp()

# plot.figure_plot()

# plot.activate_figure_option()

{}


KeyError: '/xpilot/xdebug'

In [ ]:
# bokeh test
x = np.linspace(0, 10, 50)
y = np.sin(x)
coef = [1,2,3]
source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(y_range=(-10, 10), width=400, height=400)

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6,legend_label = 'Test Line')
plot.scatter('x', 'y', source=source, size=3, color = 'green', legend_label = 'Test Point')
plot.toolbar.active_scroll = plot.select_one(WheelZoomTool)
plot.legend.click_policy = 'hide'

amp = Slider(start=0.1, end=10, value=1, step=.1, title="Amplitude")
freq = Slider(start=0.1, end=10, value=1, step=.1, title="Frequency")


callback = CustomJS(args=dict(source=source, amp=amp, freq=freq, coef = coef),
                    code="""
    const A = amp.value
    const k = freq.value


    const x = source.data.x
    const y = Array.from(x, (x) => A*Math.sin(k*x))
    source.data = { x, y }
""")

amp.js_on_change('value', callback)
freq.js_on_change('value', callback)
output_notebook()

show(row(plot, column(amp, freq)))

NameError: name 'np' is not defined